In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from astropy.table import Table
from itertools import combinations
from astropy.coordinates import SkyCoord
from astropy import units as u


save = True

sns.set(style="whitegrid")

## Example

## 📍 INPUT: Gaia-style observables

You give `SkyCoord` the usual 6D astrometric quantities:

* `ra`, `dec`: right ascension and declination (angular position on the sky)
* `parallax`: gives distance → $d = \frac{1000}{\text{parallax in mas}}$
* `pmra`, `pmdec`: proper motions (tangential angular speed)
* `radial_velocity`: line-of-sight speed

---

## 🧠 What SkyCoord Does Internally

### 1. **Position (x, y, z)**

It transforms your angular position + distance into Cartesian coordinates in the **ICRS (equatorial) frame**, centered on the Sun:

$$
\begin{aligned}
x &= d \cdot \cos(\text{dec}) \cdot \cos(\text{ra}) \\
y &= d \cdot \cos(\text{dec}) \cdot \sin(\text{ra}) \\
z &= d \cdot \sin(\text{dec})
\end{aligned}
$$

where `ra` and `dec` are in radians, and `d` is in parsecs.

### 2. **Velocity (vx, vy, vz)**

It uses the radial velocity and the proper motions to get the 3D space velocity:

* Proper motions are angular → converted into **km/s** using:

  $$
  v = 4.74047 \times \mu \times d
  $$
* It then projects these tangential and radial velocities into Cartesian components, aligned with the $x, y, z$ from above.

---

## ✅ Example:

If you do:

```python
c = SkyCoord(ra=..., dec=..., distance=..., pm_ra_cosdec=..., pm_dec=..., radial_velocity=...)
```

Then:

```python
c.cartesian.xyz  → gives (x, y, z) in pc  
c.velocity.d_xyz → gives (vx, vy, vz) in km/s
```

All in a consistent 3D Cartesian frame, with origin at the Sun and axes aligned with the ICRS system:

* $x$ toward the vernal equinox (RA=0)
* $z$ toward celestial north pole (Dec=+90°)
* $y$ completes the right-handed system

---


In [2]:
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np


# bulk motion of Pleiades clusters
def compute_bulk_motion(pleiades_stars):
    # Convert the star data to SkyCoord objects
    coords = SkyCoord(
        ra=pleiades_stars['ra'] * u.deg,
        dec=pleiades_stars['dec'] * u.deg,
        distance=(1000 / pleiades_stars['parallax']) * u.pc,
        pm_ra_cosdec=pleiades_stars['pmra'] * u.mas/u.yr,
        pm_dec=pleiades_stars['pmdec'] * u.mas/u.yr,
        radial_velocity=pleiades_stars['radial_velocity'] * u.km/u.s
    )

    # Calculate the mean proper motion and radial velocity
    mean_pm_ra = np.mean(coords.pm_ra_cosdec)
    mean_pm_dec = np.mean(coords.pm_dec)
    mean_radial_velocity = np.mean(coords.radial_velocity)

    return {
        'mean_pm_ra': mean_pm_ra.value,
        'mean_pm_dec': mean_pm_dec.value,
        'mean_radial_velocity': mean_radial_velocity.value
    }



def compute_full_orbital_elements(star1, star2):
    G = 4.302e-3  # pc * (km/s)^2 / Msun

    # Create SkyCoord objects
    c1 = SkyCoord(
        ra=star1['ra'] * u.deg,
        dec=star1['dec'] * u.deg,
        distance=(1000 / star1['parallax']) * u.pc,
        pm_ra_cosdec=star1['pmra'] * u.mas/u.yr,
        pm_dec=star1['pmdec'] * u.mas/u.yr,
        radial_velocity=star1['radial_velocity'] * u.km/u.s
    )

    c2 = SkyCoord(
        ra=star2['ra'] * u.deg,
        dec=star2['dec'] * u.deg,
        distance=(1000 / star2['parallax']) * u.pc,
        pm_ra_cosdec=star2['pmra'] * u.mas/u.yr,
        pm_dec=star2['pmdec'] * u.mas/u.yr,
        radial_velocity=star2['radial_velocity'] * u.km/u.s
    )

    r1 = c1.cartesian.xyz.to(u.pc).value
    v1 = c1.velocity.d_xyz.to(u.km/u.s).value
    r2 = c2.cartesian.xyz.to(u.pc).value
    v2 = c2.velocity.d_xyz.to(u.km/u.s).value

    r_rel = r1 - r2
    v_rel = v1 - v2
    r_mag = np.linalg.norm(r_rel)
    v_mag = np.linalg.norm(v_rel)

    M = star1['mass'] + star2['mass']

    h_vec = np.cross(r_rel, v_rel)
    h_mag = np.linalg.norm(h_vec)

    E = 0.5 * v_mag**2 - G * M / r_mag
    a = -G * M / (2 * E)

    e_vec = (np.cross(v_rel, h_vec) / G / M) - (r_rel / r_mag)
    e = np.linalg.norm(e_vec)

    i_rad = np.arccos(h_vec[2] / h_mag)
    i_deg = np.degrees(i_rad)

    # Node vector (points toward ascending node)
    K = np.array([0, 0, 1])
    n_vec = np.cross(K, h_vec)
    n_mag = np.linalg.norm(n_vec)

    # Longitude of ascending node Ω
    Omega = np.degrees(np.arccos(n_vec[0] / n_mag)) if n_mag != 0 else 0
    if n_vec[1] < 0:
        Omega = 360 - Omega

    # Argument of periapsis ω
    if n_mag != 0 and e != 0:
        omega = np.degrees(np.arccos(np.dot(n_vec, e_vec) / (n_mag * e)))
        if e_vec[2] < 0:
            omega = 360 - omega
    else:
        omega = 0

    # True anomaly ν
    if e != 0:
        nu = np.degrees(np.arccos(np.dot(e_vec, r_rel) / (e * r_mag)))
        if np.dot(r_rel, v_rel) < 0:
            nu = 360 - nu
    else:
        nu = 0

    T_yr = 2 * np.pi * np.sqrt(a**3 / (G * M))

    return {
        'semi_major_axis_pc': a,
        'eccentricity': e,
        'inclination_deg': i_deg,
        'longitude_of_ascending_node_deg': Omega,
        'argument_of_periapsis_deg': omega,
        'true_anomaly_deg': nu,
        'orbital_period_yr': T_yr
    }

# Use the same mock stars as before
example_star1 = {
    'ra': 56.75,
    'dec': 24.12,
    'parallax': 7.5,
    'pmra': 20.0,
    'pmdec': -45.0,
    'radial_velocity': 5.0,
    'mass': 0.8
}

example_star2 = {
    'ra': 56.80,
    'dec': 24.10,
    'parallax': 7.6,
    'pmra': 21.0,
    'pmdec': -46.0,
    'radial_velocity': 4.8,
    'mass': 0.6
}

full_orbit_params = compute_full_orbital_elements(example_star1, example_star2)
full_orbit_params


C:\Users\bosca\AppData\Local\Temp\ipykernel_28940\2355355480.py:103: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt(a**3 / (G * M))


{'semi_major_axis_pc': -0.018544151756192253,
 'eccentricity': 85.94435273649862,
 'inclination_deg': 144.08408655698386,
 'longitude_of_ascending_node_deg': 94.30675767904441,
 'argument_of_periapsis_deg': 20.849051466804006,
 'true_anomaly_deg': 26.525445975821114,
 'orbital_period_yr': nan}

## Let's try with our data

In [3]:
Gaia_df = pd.read_csv('DATA/My_Pleiades_filtered_Gaia_DR3_with_mass.csv')
binaries_candidates_df = pd.read_csv('DATA/Pleiades_Wide_Binary_Candidates.csv')

print(Gaia_df.columns)
print(binaries_candidates_df.columns)
binaries_candidates_df.head()

Index(['source_id', 'ra', 'dec', 'ra_error', 'dec_error', 'ra_dec_corr',
       'parallax', 'parallax_error', 'pmra', 'pmra_error', 'pmdec',
       'pmdec_error', 'phot_g_mean_mag', 'phot_bp_mean_mag',
       'phot_rp_mean_mag', 'bp_rp', 'radial_velocity', 'radial_velocity_error',
       'ipd_frac_multi_peak', 'ipd_gof_harmonic_amplitude',
       'astrometric_chi2_al', 'astrometric_n_good_obs_al', 'ruwe',
       'duplicated_source', 'Mass (M_sun)'],
      dtype='object')
Index(['index_1', 'index_2', 'source_id_1', 'source_id_2', 'rp_kAU', 'vp_kms',
       'vc_kms', 'v_ratio', 'mass1', 'mass2', 'Mtot', 'distance1_pc',
       'distance2_pc'],
      dtype='object')


,index_1,index_2,source_id_1,source_id_2,rp_kAU,vp_kms,vc_kms,v_ratio,mass1,mass2,Mtot,distance1_pc,distance2_pc
0,84,85,66480944080770176,66481734354737792,34.569017,0.473008,0.194119,2.436696,0.834,0.634,1.468,136.388227,135.670414
1,100,101,66517468482370176,66517468482370304,1.767555,0.797801,0.947425,0.842073,0.586,1.202,1.788,137.078128,137.194366
2,130,227,66565365957676416,66754241438299520,36.628056,0.812116,0.208532,3.894441,1.048,0.747,1.795,137.970735,138.443750
3,155,156,64979732749686016,64980278208557696,39.528679,2.315711,0.196383,11.791832,0.971,0.747,1.718,135.381593,135.847327
4,199,208,66715273197982848,66727234683931520,31.362666,0.559038,0.252757,2.211763,1.663,0.595,2.258,137.564596,137.421360


In [4]:
# taking the binaries candidates from the Gaia DR3
binaries_candidates_index1 = binaries_candidates_df['source_id_1'].values
binaries_candidates_index2 = binaries_candidates_df['source_id_2'].values

# Get the Gaia DR3 data for the binary candidates
binaries_1 = Gaia_df[Gaia_df['source_id'].isin(binaries_candidates_index1)]
binaries_2 = Gaia_df[Gaia_df['source_id'].isin(binaries_candidates_index2)]

# orbits parameters
orbit_params = ['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'parallax', 'radial_velocity', 'Mass (M_sun)', 'ra_error', 'dec_error', 'pmra_error', 'pmdec_error', 'parallax_error', 'radial_velocity_error']

# Get the orbit parameters for the binary candidates
binaries_members_1 = binaries_1[orbit_params].copy()
binaries_members_2 = binaries_2[orbit_params].copy()

binaries_members_1.head()

,source_id,ra,dec,pmra,pmdec,parallax,radial_velocity,Mass (M_sun),ra_error,dec_error,pmra_error,pmdec_error,parallax_error,radial_velocity_error
84,66480944080770176,57.820371,23.826400,19.840059,-46.761245,7.332011,5.824165,0.834,0.016239,0.009507,0.021189,0.012711,0.017303,0.819238
100,66517468482370176,57.075122,23.891274,19.104444,-45.348619,7.295110,8.476849,0.586,0.020812,0.014940,0.026129,0.018849,0.021686,2.436734
130,66565365957676416,57.587208,24.466545,20.345936,-45.482774,7.247914,5.401723,1.048,0.020498,0.015692,0.027922,0.022154,0.023267,0.401106
155,64979732749686016,56.756010,23.494741,19.071893,-45.023438,7.386529,4.517381,0.971,0.017084,0.013305,0.022934,0.015802,0.018657,0.662631
199,66715273197982848,56.830700,24.138920,19.017070,-43.927571,7.269312,4.730700,1.663,0.025299,0.019494,0.032082,0.024707,0.027774,0.228806


## Calculating the Bulk motion of the Pleiades

In [5]:
# Weighted mean function
def weighted_mean_and_error(values, errors):
    weights = 1 / errors**2
    return np.sum(values * weights) / np.sum(weights), np.sqrt(1 / np.sum(weights))

# Compute weighted bulk motion and error
bulk_pmra, bulk_pmra_err = weighted_mean_and_error(Gaia_df['pmra'].values, Gaia_df['pmra_error'].values)
bulk_pmdec, bulk_pmdec_err = weighted_mean_and_error(Gaia_df['pmdec'].values, Gaia_df['pmdec_error'].values)
bulk_rv, bulk_rv_err = weighted_mean_and_error(Gaia_df['radial_velocity'].values, Gaia_df['radial_velocity_error'].values)

print(f"Bulk proper motion in RA: {bulk_pmra} ± {bulk_pmra_err} mas/yr")
print(f"Bulk proper motion in Dec: {bulk_pmdec} ± {bulk_pmdec_err} mas/yr")
print(f"Bulk radial velocity: {bulk_rv} ± {bulk_rv_err} km/s")
print(f"Number of stars used: {len(Gaia_df)}")

Bulk proper motion in RA: 19.792777131936337 ± 0.0013851619235368102 mas/yr
Bulk proper motion in Dec: -45.1077640525711 ± 0.0009366106707717053 mas/yr
Bulk radial velocity: 7.29939054712946 ± 0.04118234226701103 km/s
Number of stars used: 332


In [6]:
## Subtrack the bulk motion from the binary candidates
binaries_members_1['pmra'] -= bulk_pmra
binaries_members_1['pmdec'] -= bulk_pmdec
binaries_members_1['radial_velocity'] -= bulk_rv

binaries_members_2['pmra'] -= bulk_pmra
binaries_members_2['pmdec'] -= bulk_pmdec
binaries_members_2['radial_velocity'] -= bulk_rv

# subtract the bulk motion errors
binaries_members_1['pmra_error'] = np.sqrt(binaries_members_1['pmra_error']**2 + bulk_pmra_err**2)
binaries_members_1['pmdec_error'] = np.sqrt(binaries_members_1['pmdec_error']**2 + bulk_pmdec_err**2)
binaries_members_1['radial_velocity_error'] = np.sqrt(binaries_members_1['radial_velocity_error']**2 + bulk_rv_err**2)

binaries_members_2['pmra_error'] = np.sqrt(binaries_members_2['pmra_error']**2 + bulk_pmra_err**2)
binaries_members_2['pmdec_error'] = np.sqrt(binaries_members_2['pmdec_error']**2 + bulk_pmdec_err**2)
binaries_members_2['radial_velocity_error'] = np.sqrt(binaries_members_2['radial_velocity_error']**2 + bulk_rv_err**2)

## Monte Carlo sampling for the error

Since we have only one measurament we apply a MCM sampling to retrive the values of each orbit parameters and its error